In [1]:
from requests import get
from selenium import webdriver
from bs4 import BeautifulSoup
from pandas import Series
from pandas import DataFrame
from time import time
from time import sleep
from random import randint
from IPython.core.display import clear_output
import pandas as pd
import re

url_list = []
domain = 'https://www.yelp.com'
        
list_start = 0    

for i in range(0,4):
    page_url = 'https://www.yelp.com/search?attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2&cflt=breakfast_brunch&find_desc=Breakfast+%26+Brunch&find_loc=Helsinki&start='+str(list_start)
    
    response = get(page_url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    restaurant_urls = html_soup.find_all('a', class_ = 'biz-name')

    for single_url in restaurant_urls:
        temp = domain + single_url.get('href')
        url_list.append(temp)
        
    list_start = list_start + 10
    
start_time = time()
requests = 0    
names = []
reviews = []
ratings = []
prices = []
cards = []
saturdays = []
sundays = []
distances = []
start_location = '?start=Kamppi%2C%20Helsinki%2C%20Finland'

for url in url_list:
    response = get(url)
    
    sleep(randint(8,15))
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
    
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    name = page_html.find('h1',class_='biz-page-title')    
    if name is not None:
        name = name.text.strip()
    else:
        name = 'Empty'
    names.append(name)
    
    review = page_html.find('span', class_ = 'review-count rating-qualifier')
    if review is not None:
        review = review.text.strip()
        review = int(filter(unicode.isdigit,review))
    else:
        review = 0
    reviews.append(review)
    
    rating = page_html.find('img', class_ = 'offscreen').get('alt')
    if rating is not None:
        rating = rating.strip()
        rating = rating.partition(' ')[0]
        rating = float(rating)
    else:
        rating = 0
    ratings.append(rating)
    
    price = page_html.find('dd', class_ = 'price-description')
    if price is not None: 
        price = price.text.strip()
    else:
        price = 'Empty'
    prices.append(price)
    
    card = page_html.find('div', class_ = 'short-def-list').dd
    if card is not None:
        card = card.text.strip()
    else:
        card = 'Empty'
    cards.append(card)
    
    saturday = page_html.find('th', text='Sat')
    if saturday is not None:
        saturday = saturday.find_next_sibling('td').text.strip()
    else:
        saturday = 'Empty'
    saturdays.append(saturday)
        
    sunday = page_html.find('th', text='Sun')    
    if sunday is not None:
        sunday = sunday.find_next_sibling('td').text.strip()
    else:
        sunday = 'Empty'
    
    sundays.append(sunday)
    
    map_url = domain + page_html.find('a',class_='biz-directions').get('href') + start_location
    driver = webdriver.Chrome()
    driver.get(map_url)
    
    map_page_html = BeautifulSoup(driver.page_source)
    distance = map_page_html.find('div', class_ ='adp-summary')

    if distance is not None:
        distance = distance.span.text.strip()
    else:
        distance = 'Empty'
        
    distance = distance.partition(' ')[0]
    distances.append(distance)
    driver.quit()
    
    

restaurants_data = pd.DataFrame({'Name': names,
                                 'Review counts': reviews,
                                 'Ratings': ratings,
                                 'Price range': prices,
                                 'Credit card': cards,
                                 'Saturday': saturdays,
                                 'Sunday': sundays,
                                 'Distance in KM' : distances})
print(restaurants_data.info())
restaurants_data.to_csv('yelp-data-all.csv', sep=',', encoding='utf-8')
restaurants_data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 8 columns):
Credit card       35 non-null object
Distance in KM    35 non-null object
Name              35 non-null object
Price range       35 non-null object
Ratings           35 non-null float64
Review counts     35 non-null int64
Saturday          35 non-null object
Sunday            35 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 2.3+ KB
None


,Credit card,Distance in KM,Name,Price range,Ratings,Review counts,Saturday,Sunday
0,Yes,0.8,Fleuriste,€13-21,4.5,20,10:00 am - 5:00 pm,Closed
1,Yes,0.6,SIS. Deli & Cafe,€13-21,4.5,16,9:00 am - 4:00 pm,9:00 am - 4:00 pm
2,Yes,1.9,Café Engel,€13-21,4.0,27,9:00 am - 9:00 pm,10:00 am - 7:00 pm
3,Yes,2.0,Café DaJa,€13-21,4.5,9,9:00 am - 6:00 pm,9:00 am - 6:00 pm
4,Yes,1.4,La Torrefazione,€13-21,4.0,37,9:00 am - 7:00 pm,10:00 am - 6:30 pm
5,Yes,1.6,Karl Fazer Café,€13-21,4.0,80,9:00 am - 10:00 pm,10:00 am - 6:00 pm
6,Yes,1.5,Tin Tin Tango,€13-21,4.0,27,9:00 am - 12:00 am,10:00 am - 9:00 pm
7,Yes,1.0,Cargo,€13-21,4.5,7,10:00 am - 10:00 pm,10:00 am - 4:00 pm
8,Yes,3.6,Moko Market & Cafe,€13-21,4.5,4,10:00 am - 3:00 pm,Closed
9,Yes,0.9,Deli Cafe Maya,Under €12,4.0,11,10:00 am - 4:00 pm,10:00 am - 4:00 pm
